import pandas as pd
import numpy as np
import os


In [2]:
import pandas as pd
import numpy as np
import os
from transition_analsyis_functions import (extract_and_normalize_t1_data,add_columns_from_excel)

In [3]:

# ----------------
# Paths
# ----------------
in_path = r"only_Q_outputs/combined/timepoints_file_inverted_2.csv"
excel_path = r"data\original_Q_data\Study_Questionnaire_Responses_October.xlsx"
excel_sheet = 0

out_dir = r"only_Q_outputs/combined"
os.makedirs(out_dir, exist_ok=True)
out_path = os.path.join(out_dir, "timepoints_file_inverted_2_with_meta_and_transitions.csv")

subject_col = "Subject_Code"

# ----------------
# Binary columns (0/1) used for transitions
# ----------------
binary_cols = ["b", "t1", "t2", "t3", "after"]  # <-- edit if your column names differ

# ----------------
# Load inverted file
# ----------------
df = pd.read_csv(in_path)
if subject_col not in df.columns:
    raise ValueError(f"'{subject_col}' not found in inverted file. Columns: {list(df.columns)}")

df[subject_col] = df[subject_col].astype(str)

# ----------------
# Load Excel + choose columns to add
# ----------------
meta = pd.read_excel(excel_path, sheet_name=excel_sheet)
if subject_col not in meta.columns:
    raise ValueError(f"'{subject_col}' not found in Excel file. Columns: {list(meta.columns)}")

cols_to_add = [
    "b_PHQ_total", "T1_PHQ_total", "T2_PHQ_total", "T3_PHQ_total", "after_PHQ_total",
    "b_GAD7_total", "T1_GAD7_total", "T2_GAD7_total", "T3_GAD7_total", "after_GAD7_total",
    *binary_cols,  # include the binary columns if they are in Excel
]

# keep only columns that actually exist
cols_to_add_existing = [c for c in cols_to_add if c in meta.columns]

meta_small = meta[[subject_col] + cols_to_add_existing].copy()
meta_small[subject_col] = meta_small[subject_col].astype(str)

# ----------------
# Merge meta into the full inverted file (ONE dataframe)
# ----------------
out_df = df.merge(meta_small, on=subject_col, how="left")

# ----------------
# Helper: build a transition label column
# ----------------
def add_transition_col(df_, from_col, to_col, new_col_name):
    if from_col not in df_.columns or to_col not in df_.columns:
        raise ValueError(
            f"Missing '{from_col}' or '{to_col}' for {new_col_name}. "
            f"Available columns: {list(df_.columns)}"
        )

    a = pd.to_numeric(df_[from_col], errors="coerce")
    b = pd.to_numeric(df_[to_col], errors="coerce")

    conditions = [
        (a == 0) & (b == 0),
        (a == 0) & (b == 1),
        (a == 1) & (b == 0),
        (a == 1) & (b == 1),
    ]
    choices = ["stay_good", "worsen", "improve", "stay_bad"]

    df_[new_col_name] = np.select(conditions, choices, default="missing")
    return df_

# ----------------
# Create all transition columns
# ----------------
transitions = [
    ("b", "t1", "B_TO_T1"),
    ("t1", "t2", "T1_TO_T2"),
    ("t2", "t3", "T2_TO_T3"),
    ("t3", "after", "T3_TO_AFTER"),
]

for f, t, name in transitions:
    out_df = add_transition_col(out_df, f, t, name)

# ----------------
# Save ONE file
# ----------------
out_df.to_csv(out_path, index=False)
print(f"Saved: {out_path}")


Saved: only_Q_outputs/combined\timepoints_file_inverted_2_with_meta_and_transitions.csv


In [4]:
import pandas as pd

# ----------------
# Paths
# ----------------
transitions_file_path = r"only_Q_outputs/combined/timepoints_file_inverted_2_with_meta_and_transitions.csv"

# Column names
subject_col = "Subject_Code"  # Column name for subject codes
cluster_cols = ["b", "t1", "t2", "t3", "after"]  # Cluster columns (0/1)

# ----------------
# Load transitions file
# ----------------
print(f"Loading transitions file: {transitions_file_path}")
df = pd.read_csv(transitions_file_path)

# Check if subject column exists
if subject_col not in df.columns:
    raise ValueError(f"'{subject_col}' not found in transitions file. Available columns: {list(df.columns)}")

# Check which cluster columns exist
existing_cluster_cols = [col for col in cluster_cols if col in df.columns]
missing_cluster_cols = [col for col in cluster_cols if col not in df.columns]

if missing_cluster_cols:
    print(f"Warning: These cluster columns not found: {missing_cluster_cols}")

if not existing_cluster_cols:
    raise ValueError(f"None of the cluster columns {cluster_cols} found in transitions file.")

# ----------------
# Extract subject codes and cluster columns
# ----------------
# Select only subject code and cluster columns
result_df = df[[subject_col] + existing_cluster_cols].copy()

# Convert subject code to string
result_df[subject_col] = result_df[subject_col].astype(str)

# Convert cluster columns to numeric
for col in existing_cluster_cols:
    result_df[col] = pd.to_numeric(result_df[col], errors="coerce")

# Remove rows where all cluster columns are missing
result_df = result_df.dropna(subset=existing_cluster_cols, how='all')

# ----------------
# Display results
# ----------------
print(f"\nLoaded {len(result_df)} subjects")
print(f"Subject column: {subject_col}")
print(f"Cluster columns: {existing_cluster_cols}")
print(f"\nFirst few rows:")
print(result_df.head(10))
print(f"\nData shape: {result_df.shape}")
print(f"\nColumn info:")
print(result_df.info())


Loading transitions file: only_Q_outputs/combined/timepoints_file_inverted_2_with_meta_and_transitions.csv

Loaded 109 subjects
Subject column: Subject_Code
Cluster columns: ['b', 't1', 't2', 't3', 'after']

First few rows:
  Subject_Code  b  t1  t2  t3  after
0        CT003  1   0   0   0      0
1        CT004  0   0   0   0      1
2        CT005  1   1   0   1      0
3        CT007  1   1   1   1      1
4        CT008  1   0   0   0      0
5        CT010  0   1   0   1      0
6        CT012  0   1   1   1      0
7        CT013  1   0   1   0      0
8        CT015  0   1   1   0      1
9        CT016  0   0   0   0      1

Data shape: (109, 6)

Column info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Subject_Code  109 non-null    object
 1   b             109 non-null    int64 
 2   t1            109 non-null    int64 
 3   t2            

In [5]:
import pandas as pd
import os

# ----------------
# Configuration
# ----------------
# Input file (transitions file with all timepoints)
transitions_file_path = r"only_Q_outputs/combined/timepoints_file_inverted_2_with_meta_and_transitions.csv"

# Output file
out_dir = r"only_Q_outputs/combined"
os.makedirs(out_dir, exist_ok=True)
output_file_path = os.path.join(out_dir, "subjects_after_1_with_timepoints.csv")

# Column names
subject_col = "Subject_Code"  # Subject code column name
timepoint_cols = ["b", "t1", "t2", "t3", "after"]  # Timepoint cluster columns
after_col = "after"  # Column to filter on (should be 1)

# ----------------
# Load transitions file
# ----------------
print(f"Loading transitions file: {transitions_file_path}")
df = pd.read_csv(transitions_file_path)

print(f"Loaded {len(df)} subjects")
print(f"Total columns: {len(df.columns)}")

# ----------------
# Check required columns
# ----------------
if subject_col not in df.columns:
    raise ValueError(f"'{subject_col}' not found in file. Available columns: {list(df.columns)}")

if after_col not in df.columns:
    raise ValueError(f"'{after_col}' not found in file. Available columns: {list(df.columns)}")

# Check which timepoint columns exist
existing_timepoint_cols = [col for col in timepoint_cols if col in df.columns]
missing_timepoint_cols = [col for col in timepoint_cols if col not in df.columns]

if missing_timepoint_cols:
    print(f"Warning: These timepoint columns not found: {missing_timepoint_cols}")

if not existing_timepoint_cols:
    raise ValueError(f"None of the timepoint columns {timepoint_cols} found in file.")

# ----------------
# Convert columns to numeric
# ----------------
df[after_col] = pd.to_numeric(df[after_col], errors="coerce")
df[subject_col] = df[subject_col].astype(str)

for col in existing_timepoint_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# ----------------
# Filter subjects with after = 1
# ----------------
print(f"\n{'='*70}")
print("Filtering subjects with after = 1...")
print(f"{'='*70}")

n_before = len(df)
df_filtered = df[df[after_col] == 1].copy()
n_after = len(df_filtered)

print(f"Subjects before filtering: {n_before}")
print(f"Subjects with {after_col} = 1: {n_after}")
print(f"Subjects removed: {n_before - n_after}")

# ----------------
# Select only subject code and timepoint columns
# ----------------
output_df = df_filtered[[subject_col] + existing_timepoint_cols].copy()

# Sort by subject code
output_df = output_df.sort_values(by=subject_col).reset_index(drop=True)

# ----------------
# Save output file
# ----------------
output_df.to_csv(output_file_path, index=False)

print(f"\n{'='*70}")
print(f"Output file saved: {output_file_path}")
print(f"Total subjects: {len(output_df)}")
print(f"Total columns: {len(output_df.columns)}")
print(f"Columns: {list(output_df.columns)}")
print(f"\nFirst few rows:")
print(output_df.head(15))
print(f"\nTimepoint distribution:")
for col in existing_timepoint_cols:
    if col != after_col:
        print(f"\n{col}:")
        print(output_df[col].value_counts().sort_index())

Loading transitions file: only_Q_outputs/combined/timepoints_file_inverted_2_with_meta_and_transitions.csv
Loaded 109 subjects
Total columns: 20

Filtering subjects with after = 1...
Subjects before filtering: 109
Subjects with after = 1: 33
Subjects removed: 76

Output file saved: only_Q_outputs/combined\subjects_after_1_with_timepoints.csv
Total subjects: 33
Total columns: 6
Columns: ['Subject_Code', 'b', 't1', 't2', 't3', 'after']

First few rows:
   Subject_Code  b  t1  t2  t3  after
0         CT004  0   0   0   0      1
1         CT007  1   1   1   1      1
2         CT015  0   1   1   0      1
3         CT016  0   0   0   0      1
4         CT027  0   0   1   1      1
5         CT033  0   0   0   0      1
6         CT038  0   1   0   1      1
7         CT039  1   1   1   1      1
8         CT044  1   0   1   1      1
9         CT047  0   1   1   1      1
10        CT051  1   1   1   0      1
11        CT058  0   0   0   0      1
12        NT002  1   0   0   0      1
13        NT0

In [6]:
# Configuration
columns_to_extract = [
    ("DMN_GM", "cortical_networks_full_brain"),
    ("DAN_GM", "cortical_networks_full_brain"),
    ("FPN_GM", "cortical_networks_full_brain"),
    ("LIM_GM", "cortical_networks_full_brain"),
    ("SMN_GM", "cortical_networks_full_brain"),
    ("VAN_GM", "cortical_networks_full_brain"),
    ("VIS_GM", "cortical_networks_full_brain"),
    ("etiv", "global_parameters"),
]

output_filename = "T1_networks_all_brain_all_subjects_with_after_normalized_by_etiv_NO_NAN.csv"

# Call the function
df = extract_and_normalize_t1_data(
    columns_to_extract=columns_to_extract,
    output_filename=output_filename
)

Loading transitions file: only_Q_outputs/combined/timepoints_file_inverted_2_with_meta_and_transitions.csv
Transitions subjects: 109

Processing 8 column(s) to extract...
Columns grouped into 2 sheet(s)

Processing sheet: cortical_networks_full_brain
Columns requested: ['DMN_GM', 'DAN_GM', 'FPN_GM', 'LIM_GM', 'SMN_GM', 'VAN_GM', 'VIS_GM']
Found 117 rows in this sheet
  Extracted 'DMN_GM': 117 non-null values
  Extracted 'DAN_GM': 117 non-null values
  Extracted 'FPN_GM': 117 non-null values
  Extracted 'LIM_GM': 117 non-null values
  Extracted 'SMN_GM': 117 non-null values
  Extracted 'VAN_GM': 117 non-null values
  Extracted 'VIS_GM': 117 non-null values

Processing sheet: global_parameters
Columns requested: ['etiv']
Found 117 rows in this sheet
  Extracted 'etiv': 117 non-null values

Creating merged dataframe (in-memory)...
Unique subjects in extracted T1 data: 117
After merge: rows=117, cols=10
Columns: ['subject_code', 'DMN_GM', 'DAN_GM', 'FPN_GM', 'LIM_GM', 'SMN_GM', 'VAN_GM', '

In [7]:
# Configuration
columns_to_extract = [
    ("DMN_GM", "cortical_networks_full_brain"),
    ("DAN_GM", "cortical_networks_full_brain"),
    ("FPN_GM", "cortical_networks_full_brain"),
    ("LIM_GM", "cortical_networks_full_brain"),
    ("SMN_GM", "cortical_networks_full_brain"),
    ("VAN_GM", "cortical_networks_full_brain"),
    ("VIS_GM", "cortical_networks_full_brain"),
    ("etiv", "global_parameters"),
]

output_filename = "T1_networks_all_brain_all_subjects_with_after_normalized_by_etiv_NO_NAN.csv"

# Call the function
df = extract_and_normalize_t1_data(
    columns_to_extract=columns_to_extract,
    output_filename=output_filename
)

Loading transitions file: only_Q_outputs/combined/timepoints_file_inverted_2_with_meta_and_transitions.csv
Transitions subjects: 109

Processing 8 column(s) to extract...
Columns grouped into 2 sheet(s)

Processing sheet: cortical_networks_full_brain
Columns requested: ['DMN_GM', 'DAN_GM', 'FPN_GM', 'LIM_GM', 'SMN_GM', 'VAN_GM', 'VIS_GM']
Found 117 rows in this sheet
  Extracted 'DMN_GM': 117 non-null values
  Extracted 'DAN_GM': 117 non-null values
  Extracted 'FPN_GM': 117 non-null values
  Extracted 'LIM_GM': 117 non-null values
  Extracted 'SMN_GM': 117 non-null values
  Extracted 'VAN_GM': 117 non-null values
  Extracted 'VIS_GM': 117 non-null values

Processing sheet: global_parameters
Columns requested: ['etiv']
Found 117 rows in this sheet
  Extracted 'etiv': 117 non-null values

Creating merged dataframe (in-memory)...
Unique subjects in extracted T1 data: 117
After merge: rows=117, cols=10
Columns: ['subject_code', 'DMN_GM', 'DAN_GM', 'FPN_GM', 'LIM_GM', 'SMN_GM', 'VAN_GM', '

In [8]:
# Configuration
columns_to_extract = [
    ("cingulate", "cortical_lobes_full_brain"),
    ("frontal", "cortical_lobes_full_brain"),
    ("parietal", "cortical_lobes_full_brain"),
    ("temporal", "cortical_lobes_full_brain"),
    ("occipital", "cortical_lobes_full_brain"),
    ("insula", "cortical_lobes_full_brain"),
    ("etiv", "global_parameters")
]

output_filename = "T1_lobes_all_brain_all_subjects_with_after_normalized_by_etiv_NO_NAN.csv"

# Call the function
df = extract_and_normalize_t1_data(
    columns_to_extract=columns_to_extract,
    output_filename=output_filename
)

Loading transitions file: only_Q_outputs/combined/timepoints_file_inverted_2_with_meta_and_transitions.csv
Transitions subjects: 109

Processing 7 column(s) to extract...
Columns grouped into 2 sheet(s)

Processing sheet: cortical_lobes_full_brain
Columns requested: ['cingulate', 'frontal', 'parietal', 'temporal', 'occipital', 'insula']
Found 117 rows in this sheet
  Extracted 'cingulate': 117 non-null values
  Extracted 'frontal': 117 non-null values
  Extracted 'parietal': 117 non-null values
  Extracted 'temporal': 117 non-null values
  Extracted 'occipital': 117 non-null values
  Extracted 'insula': 117 non-null values

Processing sheet: global_parameters
Columns requested: ['etiv']
Found 117 rows in this sheet
  Extracted 'etiv': 117 non-null values

Creating merged dataframe (in-memory)...
Unique subjects in extracted T1 data: 117
After merge: rows=117, cols=9
Columns: ['subject_code', 'cingulate', 'frontal', 'parietal', 'temporal', 'occipital', 'insula', 'etiv', 'after']

Normal

In [9]:
# Configuration
columns_to_extract = [
    ("cingulate_lh", "cortical_lobes_lr_"),
    ("cingulate_rh", "cortical_lobes_lr_"),
    ("frontal_lh", "cortical_lobes_lr_"),
    ("frontal_rh", "cortical_lobes_lr_"),
    ("parietal_lh", "cortical_lobes_lr_"),
    ("parietal_rh", "cortical_lobes_lr_"),
    ("temporal_lh", "cortical_lobes_lr_"),
    ("temporal_rh", "cortical_lobes_lr_"),
    ("occipital_lh", "cortical_lobes_lr_"),
    ("occipital_rh", "cortical_lobes_lr_"),
    ("insula_lh", "cortical_lobes_lr_"),
    ("insula_rh", "cortical_lobes_lr_"),
    ("etiv", "global_parameters"),
]

output_filename = "T1_lobes_lr_subjects_with_after_normalized_by_etiv_NO_NAN.csv"

# Call the function
df = extract_and_normalize_t1_data(
    columns_to_extract=columns_to_extract,
    output_filename=output_filename
)

Loading transitions file: only_Q_outputs/combined/timepoints_file_inverted_2_with_meta_and_transitions.csv
Transitions subjects: 109

Processing 13 column(s) to extract...
Columns grouped into 2 sheet(s)

Processing sheet: cortical_lobes_lr_
Columns requested: ['cingulate_lh', 'cingulate_rh', 'frontal_lh', 'frontal_rh', 'parietal_lh', 'parietal_rh', 'temporal_lh', 'temporal_rh', 'occipital_lh', 'occipital_rh', 'insula_lh', 'insula_rh']
Found 117 rows in this sheet
  Extracted 'cingulate_lh': 117 non-null values
  Extracted 'cingulate_rh': 117 non-null values
  Extracted 'frontal_lh': 117 non-null values
  Extracted 'frontal_rh': 117 non-null values
  Extracted 'parietal_lh': 117 non-null values
  Extracted 'parietal_rh': 117 non-null values
  Extracted 'temporal_lh': 117 non-null values
  Extracted 'temporal_rh': 117 non-null values
  Extracted 'occipital_lh': 117 non-null values
  Extracted 'occipital_rh': 117 non-null values
  Extracted 'insula_lh': 117 non-null values
  Extracted 'i

In [10]:
import pandas as pd
import re

# ----------------  
# Configuration
# ----------------
# Path to ttest file
ttest_file_path = r"ttest_longitude.xlsx"

# Sheet name (first sheet = index 0)
sheet_name = 0  # Change to sheet name if needed, or use 0 for first sheet

# Column name candidates for Parameter, Yeo, and Hemisphere columns
PARAMETER_COL_CANDIDATES = ["Parameter", "parameter", "STRUCT_NAME", "StructName"]
YEO_COL_CANDIDATES = ["Yeo", "YEO", "yeo", "Yeo_Network", "Network"]
HEMISPHERE_COL_CANDIDATES = ["Hemisphere", "hemisphere", "hemishpere", "Hemi", "hemi"]

# Specify which networks you want to filter
desired_networks = ["Frontoparietal", "Default Mode","Somatomotor","Limbic","Visual","Dorsal Attention","Ventral Attention"]  # CHANGE THIS to your desired networks
# Examples: ["DMN_GM", "DAN_GM", "FPN_GM", "LIM_GM", "SMN_GM", "VAN_GM", "VIS_GM"]

# ----------------  
# Helper function to find column names (case-insensitive)
# ----------------
def pick_column(df, candidates):
    """Return the first matching column name in df (case-insensitive), else None."""
    cols_map = {c.lower(): c for c in df.columns}
    for c in candidates:
        if c.lower() in cols_map:
            return cols_map[c.lower()]
    # try partial matches
    for c in candidates:
        for k, v in cols_map.items():
            if c.lower() in k:
                return v
    return None

# ----------------  
# Load ttest file
# ----------------
print(f"Loading ttest file: {ttest_file_path}")
print(f"Sheet: {sheet_name} (first sheet)")

df = pd.read_excel(ttest_file_path, sheet_name=sheet_name)

print(f"\nLoaded {len(df)} rows")
print(f"Columns: {list(df.columns)}")

# ----------------  
# Find Parameter, Yeo, and Hemisphere columns
# ----------------
param_col = pick_column(df, PARAMETER_COL_CANDIDATES)
yeo_col = pick_column(df, YEO_COL_CANDIDATES)
hemisphere_col = pick_column(df, HEMISPHERE_COL_CANDIDATES)

if not param_col:
    print(f"\nError: Could not find Parameter column.")
    print(f"Available columns: {list(df.columns)}")
    raise ValueError("Parameter column not found")

if not yeo_col:
    print(f"\nError: Could not find Yeo/Network column.")
    print(f"Available columns: {list(df.columns)}")
    raise ValueError("Yeo column not found")

if not hemisphere_col:
    print(f"\nError: Could not find Hemisphere column.")
    print(f"Available columns: {list(df.columns)}")
    raise ValueError("Hemisphere column not found")

print(f"\nFound columns:")
print(f"  Parameter column: '{param_col}'")
print(f"  Yeo/Network column: '{yeo_col}'")
print(f"  Hemisphere column: '{hemisphere_col}'")

# ----------------  
# Filter by desired networks
# ----------------
print(f"\n{'='*70}")
print(f"Filtering for networks: {desired_networks}")
print(f"{'='*70}")

# Convert to string and check for matches (case-insensitive, partial match)
df[yeo_col] = df[yeo_col].astype(str)
df[param_col] = df[param_col].astype(str)

# Filter rows where Yeo column matches any of the desired networks
filtered_df = df[df[yeo_col].isin(desired_networks)].copy()

if len(filtered_df) == 0:
    print(f"\nWarning: No parameters found for networks: {desired_networks}")
    print(f"\nAvailable networks in file:")
    unique_networks = df[yeo_col].unique()
    for net in sorted(unique_networks):
        count = len(df[df[yeo_col] == net])
        print(f"  {net}: {count} parameters")
else:
    print(f"\nFound {len(filtered_df)} parameters in specified networks")

# ----------------  
# Use hemisphere column directly from the file
# ----------------
# The hemisphere information is already in the file
# Just rename for consistency and clean up
if hemisphere_col != 'Hemisphere':
    filtered_df = filtered_df.rename(columns={hemisphere_col: 'Hemisphere'})
# Convert to string and clean up
filtered_df['Hemisphere'] = filtered_df['Hemisphere'].astype(str).str.strip()

# ----------------  
# Display results
# ----------------
print(f"\n{'='*70}")
print(f"PARAMETERS IN SPECIFIED NETWORKS")
print(f"{'='*70}")

if len(filtered_df) > 0:
    # Group by network for better organization
    for network in desired_networks:
        network_params = filtered_df[filtered_df[yeo_col] == network]
        if len(network_params) > 0:
            print(f"\nNetwork: {network} ({len(network_params)} parameters)")
            print("-" * 70)
            
            for idx, row in network_params.iterrows():
                param = row[param_col]
                hemi = row['Hemisphere']
                print(f"  {param:<50} | Hemisphere: {hemi}")
        else:
            print(f"\nNetwork: {network} - No parameters found")
    
    # Summary by hemisphere
    print(f"\n{'='*70}")
    print("SUMMARY BY HEMISPHERE")
    print(f"{'='*70}")
    hemi_counts = filtered_df['Hemisphere'].value_counts()
    for hemi, count in hemi_counts.items():
        print(f"  {hemi.capitalize()}: {count} parameters")
    
    # Summary by network
    print(f"\n{'='*70}")
    print("SUMMARY BY NETWORK")
    print(f"{'='*70}")
    network_counts = filtered_df[yeo_col].value_counts()
    for net, count in network_counts.items():
        print(f"  {net}: {count} parameters")
    
    # Create a formatted table
    print(f"\n{'='*70}")
    print("FORMATTED TABLE")
    print(f"{'='*70}")
    
    # Select relevant columns for display
    display_df = filtered_df[[param_col, yeo_col, 'Hemisphere']].copy()
    display_df = display_df.sort_values([yeo_col, 'Hemisphere', param_col]).reset_index(drop=True)
    
    print(display_df.to_string(index=False))
    
    # Show all unique parameters (for easy copying)
    print(f"\n{'='*70}")
    print("ALL PARAMETER NAMES (for easy copying):")
    print(f"{'='*70}")
    all_params = sorted(filtered_df[param_col].unique())
    for i, param in enumerate(all_params, 1):
        print(f"{i}. {param}")
        
else:
    print(f"\nNo parameters found. Available networks in file:")
    unique_networks = df[yeo_col].unique()
    for net in sorted(unique_networks):
        count = len(df[df[yeo_col] == net])
        print(f"  {net}: {count} parameters")

Loading ttest file: ttest_longitude.xlsx
Sheet: 0 (first sheet)

Loaded 68 rows
Columns: ['Parameter', 'Lobe', 'Yeo', 'Hemisphere', 'Mean_ses1', 'Mean_ses2', 'delta', 't', 'p', 'df', 'Cohen_d', 'p_corrected', 'Significant_FDR']

Found columns:
  Parameter column: 'Parameter'
  Yeo/Network column: 'Yeo'
  Hemisphere column: 'Hemisphere'

Filtering for networks: ['Frontoparietal', 'Default Mode', 'Somatomotor', 'Limbic', 'Visual', 'Dorsal Attention', 'Ventral Attention']

Found 68 parameters in specified networks

PARAMETERS IN SPECIFIED NETWORKS

Network: Frontoparietal (12 parameters)
----------------------------------------------------------------------
  bankssts                                           | Hemisphere: lh
  bankssts                                           | Hemisphere: rh
  caudalmiddlefrontal                                | Hemisphere: lh
  caudalmiddlefrontal                                | Hemisphere: rh
  fusiform                                           | He

In [11]:
# ----------------  
# Generate tuples list from filtered_df
# ----------------

# Sheet name to use (modify this based on your actual sheet name)
SHEET_NAME = "cortical_vol_lr"  # CHANGE THIS to your actual sheet name

# Generate tuples: (parameter_hemisphere, sheet_name)
tuples_list = []

for idx, row in filtered_df.iterrows():
    param = row[param_col]
    hemi = row['Hemisphere'].lower()  # Ensure lowercase (lh/rh)
    param_hemisphere = f"{param}_{hemi}"
    tuples_list.append((param_hemisphere, SHEET_NAME))

print(f"Generated {len(tuples_list)} tuples:")
print("\nAll tuples:")
for i, tup in enumerate(tuples_list, 1):
    print(f"  {i}. {tup}")

# Display as a Python list you can copy
print(f"\n{'='*70}")
print("TUPLES LIST (Python format - copy this):")
print(f"{'='*70}")
print("tuples_list = [")
for tup in tuples_list:
    print(f"    {tup},")
print("]")

# Also print just the list for easy copying
print(f"\n{'='*70}")
print("Just the list:")
print(f"{'='*70}")
print(tuples_list)

Generated 68 tuples:

All tuples:
  1. ('bankssts_lh', 'cortical_vol_lr')
  2. ('bankssts_rh', 'cortical_vol_lr')
  3. ('caudalanteriorcingulate_lh', 'cortical_vol_lr')
  4. ('caudalanteriorcingulate_rh', 'cortical_vol_lr')
  5. ('caudalmiddlefrontal_lh', 'cortical_vol_lr')
  6. ('caudalmiddlefrontal_rh', 'cortical_vol_lr')
  7. ('cuneus_lh', 'cortical_vol_lr')
  8. ('cuneus_rh', 'cortical_vol_lr')
  9. ('entorhinal_lh', 'cortical_vol_lr')
  10. ('entorhinal_rh', 'cortical_vol_lr')
  11. ('frontalpole_lh', 'cortical_vol_lr')
  12. ('frontalpole_rh', 'cortical_vol_lr')
  13. ('fusiform_lh', 'cortical_vol_lr')
  14. ('fusiform_rh', 'cortical_vol_lr')
  15. ('inferiorparietal_lh', 'cortical_vol_lr')
  16. ('inferiorparietal_rh', 'cortical_vol_lr')
  17. ('inferiortemporal_lh', 'cortical_vol_lr')
  18. ('inferiortemporal_rh', 'cortical_vol_lr')
  19. ('insula_lh', 'cortical_vol_lr')
  20. ('insula_rh', 'cortical_vol_lr')
  21. ('isthmuscingulate_lh', 'cortical_vol_lr')
  22. ('isthmuscingu

In [12]:
# Configuration
columns_to_extract = [ 
    ('bankssts_lh', 'cortical_vol_lr'),
    ('bankssts_rh', 'cortical_vol_lr'),
    ('caudalanteriorcingulate_lh', 'cortical_vol_lr'),
    ('caudalanteriorcingulate_rh', 'cortical_vol_lr'),
    ('caudalmiddlefrontal_lh', 'cortical_vol_lr'),
    ('caudalmiddlefrontal_rh', 'cortical_vol_lr'),
    ('cuneus_lh', 'cortical_vol_lr'),
    ('cuneus_rh', 'cortical_vol_lr'),
    ('entorhinal_lh', 'cortical_vol_lr'),
    ('entorhinal_rh', 'cortical_vol_lr'),
    ('frontalpole_lh', 'cortical_vol_lr'),
    ('frontalpole_rh', 'cortical_vol_lr'),
    ('fusiform_lh', 'cortical_vol_lr'),
    ('fusiform_rh', 'cortical_vol_lr'),
    ('inferiorparietal_lh', 'cortical_vol_lr'),
    ('inferiorparietal_rh', 'cortical_vol_lr'),
    ('inferiortemporal_lh', 'cortical_vol_lr'),
    ('inferiortemporal_rh', 'cortical_vol_lr'),
    ('insula_lh', 'cortical_vol_lr'),
    ('insula_rh', 'cortical_vol_lr'),
    ('isthmuscingulate_lh', 'cortical_vol_lr'),
    ('isthmuscingulate_rh', 'cortical_vol_lr'),
    ('lateraloccipital_lh', 'cortical_vol_lr'),
    ('lateraloccipital_rh', 'cortical_vol_lr'),
    ('lateralorbitofrontal_lh', 'cortical_vol_lr'),
    ('lateralorbitofrontal_rh', 'cortical_vol_lr'),
    ('lingual_lh', 'cortical_vol_lr'),
    ('lingual_rh', 'cortical_vol_lr'),
    ('medialorbitofrontal_lh', 'cortical_vol_lr'),
    ('medialorbitofrontal_rh', 'cortical_vol_lr'),
    ('middletemporal_lh', 'cortical_vol_lr'),
    ('middletemporal_rh', 'cortical_vol_lr'),
    ('paracentral_lh', 'cortical_vol_lr'),
    ('paracentral_rh', 'cortical_vol_lr'),
    ('parahippocampal_lh', 'cortical_vol_lr'),
    ('parahippocampal_rh', 'cortical_vol_lr'),
    ('parsopercularis_lh', 'cortical_vol_lr'),
    ('parsopercularis_rh', 'cortical_vol_lr'),
    ('parsorbitalis_lh', 'cortical_vol_lr'),
    ('parsorbitalis_rh', 'cortical_vol_lr'),
    ('parstriangularis_lh', 'cortical_vol_lr'),
    ('parstriangularis_rh', 'cortical_vol_lr'),
    ('pericalcarine_lh', 'cortical_vol_lr'),
    ('pericalcarine_rh', 'cortical_vol_lr'),
    ('postcentral_lh', 'cortical_vol_lr'),
    ('postcentral_rh', 'cortical_vol_lr'),
    ('posteriorcingulate_lh', 'cortical_vol_lr'),
    ('posteriorcingulate_rh', 'cortical_vol_lr'),
    ('precentral_lh', 'cortical_vol_lr'),
    ('precentral_rh', 'cortical_vol_lr'),
    ('precuneus_lh', 'cortical_vol_lr'),
    ('precuneus_rh', 'cortical_vol_lr'),
    ('rostralanteriorcingulate_lh', 'cortical_vol_lr'),
    ('rostralanteriorcingulate_rh', 'cortical_vol_lr'),
    ('rostralmiddlefrontal_lh', 'cortical_vol_lr'),
    ('rostralmiddlefrontal_rh', 'cortical_vol_lr'),
    ('superiorfrontal_lh', 'cortical_vol_lr'),
    ('superiorfrontal_rh', 'cortical_vol_lr'),
    ('superiorparietal_lh', 'cortical_vol_lr'),
    ('superiorparietal_rh', 'cortical_vol_lr'),
    ('superiortemporal_lh', 'cortical_vol_lr'),
    ('superiortemporal_rh', 'cortical_vol_lr'),
    ('supramarginal_lh', 'cortical_vol_lr'),
    ('supramarginal_rh', 'cortical_vol_lr'),
    ('temporalpole_lh', 'cortical_vol_lr'),
    ('temporalpole_rh', 'cortical_vol_lr'),
    ('transversetemporal_lh', 'cortical_vol_lr'),
    ('transversetemporal_rh', 'cortical_vol_lr'),
    ("etiv", "global_parameters")
]

output_filename = "T1_networks_parcels_68_brain_all_subjects_with_after_normalized_by_etiv_NO_NAN.csv"

# Call the function
df = extract_and_normalize_t1_data(
    columns_to_extract=columns_to_extract,
    output_filename=output_filename
)

Loading transitions file: only_Q_outputs/combined/timepoints_file_inverted_2_with_meta_and_transitions.csv
Transitions subjects: 109

Processing 69 column(s) to extract...
Columns grouped into 2 sheet(s)

Processing sheet: cortical_vol_lr
Columns requested: ['bankssts_lh', 'bankssts_rh', 'caudalanteriorcingulate_lh', 'caudalanteriorcingulate_rh', 'caudalmiddlefrontal_lh', 'caudalmiddlefrontal_rh', 'cuneus_lh', 'cuneus_rh', 'entorhinal_lh', 'entorhinal_rh', 'frontalpole_lh', 'frontalpole_rh', 'fusiform_lh', 'fusiform_rh', 'inferiorparietal_lh', 'inferiorparietal_rh', 'inferiortemporal_lh', 'inferiortemporal_rh', 'insula_lh', 'insula_rh', 'isthmuscingulate_lh', 'isthmuscingulate_rh', 'lateraloccipital_lh', 'lateraloccipital_rh', 'lateralorbitofrontal_lh', 'lateralorbitofrontal_rh', 'lingual_lh', 'lingual_rh', 'medialorbitofrontal_lh', 'medialorbitofrontal_rh', 'middletemporal_lh', 'middletemporal_rh', 'paracentral_lh', 'paracentral_rh', 'parahippocampal_lh', 'parahippocampal_rh', 'parsop

In [13]:
# Configuration
columns_to_extract = [
    ('3rd_ventricle', 'subcortical_vol'),
    ('4th_ventricle', 'subcortical_vol'),
    ('5th_ventricle', 'subcortical_vol'),
    ('brain_stem', 'subcortical_vol'),
    ('cc_anterior', 'subcortical_vol'),
    ('cc_central', 'subcortical_vol'),
    ('cc_mid_anterior', 'subcortical_vol'),
    ('cc_mid_posterior', 'subcortical_vol'),
    ('cc_posterior', 'subcortical_vol'),
    ('csf', 'subcortical_vol'),
    ('left_accumbens_area', 'subcortical_vol'),
    ('left_amygdala', 'subcortical_vol'),
    ('left_caudate', 'subcortical_vol'),
    ('left_cerebellum_cortex', 'subcortical_vol'),
    ('left_cerebellum_white_matter', 'subcortical_vol'),
    ('left_choroid_plexus', 'subcortical_vol'),
    ('left_hippocampus', 'subcortical_vol'),
    ('left_inf_lat_vent', 'subcortical_vol'),
    ('left_lateral_ventricle', 'subcortical_vol'),
    ('left_non_wm_hypointensities', 'subcortical_vol'),
    ('left_pallidum', 'subcortical_vol'),
    ('left_putamen', 'subcortical_vol'),
    ('left_thalamus', 'subcortical_vol'),
    ('left_ventraldc', 'subcortical_vol'),
    ('left_vessel', 'subcortical_vol'),
    ('left_wm_hypointensities', 'subcortical_vol'),
    ('non_wm_hypointensities', 'subcortical_vol'),
    ('optic_chiasm', 'subcortical_vol'),
    ('right_accumbens_area', 'subcortical_vol'),
    ('right_amygdala', 'subcortical_vol'),
    ('right_caudate', 'subcortical_vol'),
    ('right_cerebellum_cortex', 'subcortical_vol'),
    ('right_cerebellum_white_matter', 'subcortical_vol'),
    ('right_choroid_plexus', 'subcortical_vol'),
    ('right_hippocampus', 'subcortical_vol'),
    ('right_inf_lat_vent', 'subcortical_vol'),
    ('right_lateral_ventricle', 'subcortical_vol'),
    ('right_non_wm_hypointensities', 'subcortical_vol'),
    ('right_pallidum', 'subcortical_vol'),
    ('right_putamen', 'subcortical_vol'),
    ('right_thalamus', 'subcortical_vol'),
    ('right_ventraldc', 'subcortical_vol'),
    ('right_vessel', 'subcortical_vol'),
    ('right_wm_hypointensities', 'subcortical_vol'),
    ('wm_hypointensities', 'subcortical_vol'),
    ("etiv", "global_parameters"),
]

output_filename = "T1_subcortex_all_brain_all_subjects_with_after_normalized_by_etiv_NO_NAN.csv"

# Call the function
df = extract_and_normalize_t1_data(
    columns_to_extract=columns_to_extract,
    output_filename=output_filename
)

Loading transitions file: only_Q_outputs/combined/timepoints_file_inverted_2_with_meta_and_transitions.csv
Transitions subjects: 109

Processing 46 column(s) to extract...
Columns grouped into 2 sheet(s)

Processing sheet: subcortical_vol
Columns requested: ['3rd_ventricle', '4th_ventricle', '5th_ventricle', 'brain_stem', 'cc_anterior', 'cc_central', 'cc_mid_anterior', 'cc_mid_posterior', 'cc_posterior', 'csf', 'left_accumbens_area', 'left_amygdala', 'left_caudate', 'left_cerebellum_cortex', 'left_cerebellum_white_matter', 'left_choroid_plexus', 'left_hippocampus', 'left_inf_lat_vent', 'left_lateral_ventricle', 'left_non_wm_hypointensities', 'left_pallidum', 'left_putamen', 'left_thalamus', 'left_ventraldc', 'left_vessel', 'left_wm_hypointensities', 'non_wm_hypointensities', 'optic_chiasm', 'right_accumbens_area', 'right_amygdala', 'right_caudate', 'right_cerebellum_cortex', 'right_cerebellum_white_matter', 'right_choroid_plexus', 'right_hippocampus', 'right_inf_lat_vent', 'right_later

In [14]:
# Configuration
columns_to_extract = [
    ('3rd_ventricle', 'subcortical_vol'),
    ('4th_ventricle', 'subcortical_vol'),
    ('5th_ventricle', 'subcortical_vol'),
    ('brain_stem', 'subcortical_vol'),
    ('cc_anterior', 'subcortical_vol'),
    ('cc_central', 'subcortical_vol'),
    ('cc_mid_anterior', 'subcortical_vol'),
    ('cc_mid_posterior', 'subcortical_vol'),
    ('cc_posterior', 'subcortical_vol'),
    ('csf', 'subcortical_vol'),
    ('left_accumbens_area', 'subcortical_vol'),
    ('left_amygdala', 'subcortical_vol'),
    ('left_caudate', 'subcortical_vol'),
    ('left_cerebellum_cortex', 'subcortical_vol'),
    ('left_cerebellum_white_matter', 'subcortical_vol'),
    ('left_choroid_plexus', 'subcortical_vol'),
    ('left_hippocampus', 'subcortical_vol'),
    ('left_inf_lat_vent', 'subcortical_vol'),
    ('left_lateral_ventricle', 'subcortical_vol'),
    ('left_non_wm_hypointensities', 'subcortical_vol'),
    ('left_pallidum', 'subcortical_vol'),
    ('left_putamen', 'subcortical_vol'),
    ('left_thalamus', 'subcortical_vol'),
    ('left_ventraldc', 'subcortical_vol'),
    ('left_vessel', 'subcortical_vol'),
    ('left_wm_hypointensities', 'subcortical_vol'),
    ('non_wm_hypointensities', 'subcortical_vol'),
    ('optic_chiasm', 'subcortical_vol'),
    ('right_accumbens_area', 'subcortical_vol'),
    ('right_amygdala', 'subcortical_vol'),
    ('right_caudate', 'subcortical_vol'),
    ('right_cerebellum_cortex', 'subcortical_vol'),
    ('right_cerebellum_white_matter', 'subcortical_vol'),
    ('right_choroid_plexus', 'subcortical_vol'),
    ('right_hippocampus', 'subcortical_vol'),
    ('right_inf_lat_vent', 'subcortical_vol'),
    ('right_lateral_ventricle', 'subcortical_vol'),
    ('right_non_wm_hypointensities', 'subcortical_vol'),
    ('right_pallidum', 'subcortical_vol'),
    ('right_putamen', 'subcortical_vol'),
    ('right_thalamus', 'subcortical_vol'),
    ('right_ventraldc', 'subcortical_vol'),
    ('right_vessel', 'subcortical_vol'),
    ('right_wm_hypointensities', 'subcortical_vol'),
    ('wm_hypointensities', 'subcortical_vol'),('bankssts_lh', 'cortical_vol_lr'),
    ('bankssts_rh', 'cortical_vol_lr'),
    ('caudalanteriorcingulate_lh', 'cortical_vol_lr'),
    ('caudalanteriorcingulate_rh', 'cortical_vol_lr'),
    ('caudalmiddlefrontal_lh', 'cortical_vol_lr'),
    ('caudalmiddlefrontal_rh', 'cortical_vol_lr'),
    ('cuneus_lh', 'cortical_vol_lr'),
    ('cuneus_rh', 'cortical_vol_lr'),
    ('entorhinal_lh', 'cortical_vol_lr'),
    ('entorhinal_rh', 'cortical_vol_lr'),
    ('frontalpole_lh', 'cortical_vol_lr'),
    ('frontalpole_rh', 'cortical_vol_lr'),
    ('fusiform_lh', 'cortical_vol_lr'),
    ('fusiform_rh', 'cortical_vol_lr'),
    ('inferiorparietal_lh', 'cortical_vol_lr'),
    ('inferiorparietal_rh', 'cortical_vol_lr'),
    ('inferiortemporal_lh', 'cortical_vol_lr'),
    ('inferiortemporal_rh', 'cortical_vol_lr'),
    ('insula_lh', 'cortical_vol_lr'),
    ('insula_rh', 'cortical_vol_lr'),
    ('isthmuscingulate_lh', 'cortical_vol_lr'),
    ('isthmuscingulate_rh', 'cortical_vol_lr'),
    ('lateraloccipital_lh', 'cortical_vol_lr'),
    ('lateraloccipital_rh', 'cortical_vol_lr'),
    ('lateralorbitofrontal_lh', 'cortical_vol_lr'),
    ('lateralorbitofrontal_rh', 'cortical_vol_lr'),
    ('lingual_lh', 'cortical_vol_lr'),
    ('lingual_rh', 'cortical_vol_lr'),
    ('medialorbitofrontal_lh', 'cortical_vol_lr'),
    ('medialorbitofrontal_rh', 'cortical_vol_lr'),
    ('middletemporal_lh', 'cortical_vol_lr'),
    ('middletemporal_rh', 'cortical_vol_lr'),
    ('paracentral_lh', 'cortical_vol_lr'),
    ('paracentral_rh', 'cortical_vol_lr'),
    ('parahippocampal_lh', 'cortical_vol_lr'),
    ('parahippocampal_rh', 'cortical_vol_lr'),
    ('parsopercularis_lh', 'cortical_vol_lr'),
    ('parsopercularis_rh', 'cortical_vol_lr'),
    ('parsorbitalis_lh', 'cortical_vol_lr'),
    ('parsorbitalis_rh', 'cortical_vol_lr'),
    ('parstriangularis_lh', 'cortical_vol_lr'),
    ('parstriangularis_rh', 'cortical_vol_lr'),
    ('pericalcarine_lh', 'cortical_vol_lr'),
    ('pericalcarine_rh', 'cortical_vol_lr'),
    ('postcentral_lh', 'cortical_vol_lr'),
    ('postcentral_rh', 'cortical_vol_lr'),
    ('posteriorcingulate_lh', 'cortical_vol_lr'),
    ('posteriorcingulate_rh', 'cortical_vol_lr'),
    ('precentral_lh', 'cortical_vol_lr'),
    ('precentral_rh', 'cortical_vol_lr'),
    ('precuneus_lh', 'cortical_vol_lr'),
    ('precuneus_rh', 'cortical_vol_lr'),
    ('rostralanteriorcingulate_lh', 'cortical_vol_lr'),
    ('rostralanteriorcingulate_rh', 'cortical_vol_lr'),
    ('rostralmiddlefrontal_lh', 'cortical_vol_lr'),
    ('rostralmiddlefrontal_rh', 'cortical_vol_lr'),
    ('superiorfrontal_lh', 'cortical_vol_lr'),
    ('superiorfrontal_rh', 'cortical_vol_lr'),
    ('superiorparietal_lh', 'cortical_vol_lr'),
    ('superiorparietal_rh', 'cortical_vol_lr'),
    ('superiortemporal_lh', 'cortical_vol_lr'),
    ('superiortemporal_rh', 'cortical_vol_lr'),
    ('supramarginal_lh', 'cortical_vol_lr'),
    ('supramarginal_rh', 'cortical_vol_lr'),
    ('temporalpole_lh', 'cortical_vol_lr'),
    ('temporalpole_rh', 'cortical_vol_lr'),
    ('transversetemporal_lh', 'cortical_vol_lr'),
    ('transversetemporal_rh', 'cortical_vol_lr'),
    ("etiv", "global_parameters"),
]

output_filename = "T1_subcortex_and_cortex_all_subjects_with_after_normalized_by_etiv_NO_NAN.csv"

# Call the function
df = extract_and_normalize_t1_data(
    columns_to_extract=columns_to_extract,
    output_filename=output_filename
)

Loading transitions file: only_Q_outputs/combined/timepoints_file_inverted_2_with_meta_and_transitions.csv
Transitions subjects: 109

Processing 114 column(s) to extract...
Columns grouped into 3 sheet(s)

Processing sheet: subcortical_vol
Columns requested: ['3rd_ventricle', '4th_ventricle', '5th_ventricle', 'brain_stem', 'cc_anterior', 'cc_central', 'cc_mid_anterior', 'cc_mid_posterior', 'cc_posterior', 'csf', 'left_accumbens_area', 'left_amygdala', 'left_caudate', 'left_cerebellum_cortex', 'left_cerebellum_white_matter', 'left_choroid_plexus', 'left_hippocampus', 'left_inf_lat_vent', 'left_lateral_ventricle', 'left_non_wm_hypointensities', 'left_pallidum', 'left_putamen', 'left_thalamus', 'left_ventraldc', 'left_vessel', 'left_wm_hypointensities', 'non_wm_hypointensities', 'optic_chiasm', 'right_accumbens_area', 'right_amygdala', 'right_caudate', 'right_cerebellum_cortex', 'right_cerebellum_white_matter', 'right_choroid_plexus', 'right_hippocampus', 'right_inf_lat_vent', 'right_late

In [ ]:
# Add columns from Excel file to T1 Networks CSV
# TODO: Replace the columns_to_add list with the actual column names you want to add

excel_file_path = r"fmri_T1_clinical_with_clusters_and_pca_seeds_efa_1112 (2).xlsx"
csv_file_path = r"only_Q_outputs/combined/T1_networks_all_brain_all_subjects_with_after_normalized_by_etiv_NO_NAN.csv"

# Specify which columns to add from the Excel file
columns_to_add = [
"after_PHQ_total",'after_GAD7_total','after_DES_total','after_DERS_total'
]

# Call the function
df_updated = add_columns_from_excel(
    csv_file_path=csv_file_path,
    excel_file_path=excel_file_path,
    columns_to_add=columns_to_add,
    subject_col_csv="subject_code",
    subject_col_excel=None,  # Will auto-detect
    output_suffix="_with_extra_columns",
    verbose=True
)



Processing: T1_networks_all_brain_all_subjects_with_after_normalized_by_etiv_NO_NAN.csv
Loaded CSV: 76 rows, 9 columns
CSV columns: ['subject_code', 'after', 'DMN_GM', 'DAN_GM', 'FPN_GM', 'LIM_GM', 'SMN_GM', 'VAN_GM', 'VIS_GM']

Loaded Excel: 117 rows, 298 columns
Excel columns (first 30): ['Subject_Code', 'age_at_baseline ', 'b_ctq_total', 'b_ctq_cutoff', 'b_ctq_NEGLECT', 'b_ctq_ABUSE  ', 'b_ctq_sexual_abuse', 'b_ctq_physical_abuse', 'b_ctq_emotional_abuse', 'b_ctq_physical_neglect', 'b_ctq_emotional_neglect', 'b_ctq_sexual_abuse_cutoff', 'b_ctq_physical_abuse_cutoff', 'b_ctq_emotional_abuse_cutoff', 'b_ctq_physical_neglect_cutoff', 'b_ctq_emotional_neglect_cutoff', 'b_ctq_denial _score', 'b_lec_0_to_16_total', 'b_lec_interpersonal_events', 'b_lec_non_interpersonal_events', 'b_strength_average', 'b_PHQ_total', 'b_GAD7_total', 'b_social_support_total', 'b_pcl_total', 'b_DERS_total', 'b_DERS_Nonacceptance_Emotional_Responses', 'b_DERS_Goal_Directed_Behavior', 'b_DERS_Impulse_Control', 